# Data Structures


In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group
from collections import namedtuple, deque
import json

## 1. Basic Data Structures: Beyond Print Statements

Let's start with common Python data structures and see how sPyTial reveals their organization.

In [2]:
# Traditional approach: nested dictionaries and lists
application_data = {
    "users": [
        {"name": "Alice", "age": 30, "skills": ["Python", "JavaScript"]},
        {"name": "Bob", "age": 25, "skills": ["Java", "C++"]}
    ],
    "projects": {
        "web_app": {"lead": "Alice", "status": "active"},
        "mobile_app": {"lead": "Bob", "status": "planning"}
    },
    "stats": {"total_users": 2, "active_projects": 1}
}

print("Traditional output (try to see the structure):")
print(json.dumps(application_data, indent=2)[:200] + "...")

print("\nsPyTial spatial view:")
diagram(application_data)

Traditional output (try to see the structure):
{
  "users": [
    {
      "name": "Alice",
      "age": 30,
      "skills": [
        "Python",
        "JavaScript"
      ]
    },
    {
      "name": "Bob",
      "age": 25,
      "skills": [
     ...

sPyTial spatial view:


## 2. Binary Trees: From ASCII Art to Spatial Relations

Binary trees are perfect for demonstrating sPyTial's power. Traditional approaches use ASCII art or recursive printing that breaks down with larger trees.

In [3]:
# Define a spatially-aware binary tree
@orientation(selector='{ x : TreeNode, y : TreeNode | x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x : TreeNode, y : TreeNode | x.right = y}', directions=['below', 'right'])
@attribute(field='value')
@hideAtom(selector='NoneType')
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode({self.value})"
    
    def traditional_print(self, level=0):
        """Traditional ASCII tree printing"""
        if self.right:
            self.right.traditional_print(level + 1)
        print('    ' * level + str(self.value))
        if self.left:
            self.left.traditional_print(level + 1)
    
# Create a sample binary search tree
root = TreeNode(
    value=10,
    left=TreeNode(
        value=5,
        left=TreeNode(3),
        right=TreeNode(7)
    ),
    right=TreeNode(
        value=15,
        left=TreeNode(12),
        right=TreeNode(18)
    )
)

print("Traditional ASCII output:")
root.traditional_print()

print("\nsPyTial spatial visualization:")
diagram(root)

Traditional ASCII output:
        18
    15
        12
10
        7
    5
        3

sPyTial spatial visualization:


##  Red-Black Trees with Color Coding

Complex data structures like Red-Black trees have additional properties that traditional visualization can't easily show.

In [ ]:
# Red-Black Tree with color annotations
@atomColor(selector='{ x : RBTreeNode | @:(x.color) = red }', value='red')
@atomColor(selector='{ x : RBTreeNode | @:(x.color) = black }', value='black')
@attribute(field='value')
class RBTreeNode(TreeNode):
    def __init__(self, value, color, left=None, right=None):
        self.color = color
        super().__init__(value, left, right)

    
# Create a Red-Black Tree
rb_root = RBTreeNode(
    value=10, color="black",
    left=RBTreeNode(
        value=5, color="red",
        left=RBTreeNode(3, "black"),
        right=RBTreeNode(7, "black")
    ),
    right=RBTreeNode(
        value=15, color="red",
        left=RBTreeNode(12, "black"),
        right=RBTreeNode(18, "black")
    )
)

diagram(rb_root, method="browser")

'/var/folders/80/rtptthbx3zq0tb06wwzmck_40000gq/T/tmpl33aipj4.html'